# The Curation Station
By Xristos Katsaros

## Background:
21st century technology has been disrupting industries for better or worse. Within the music industry, we went from selling CD's, to selling MP3's, and now streaming--and it all happened very quickly. As of last year, 75% of music revenue came from streaming. Yet, 87% of the content on Spotify comes from the world's largest four record labels. And the Spotify recommendation system works in a way that will show us artists within that same 87%.

## The question:
What is a genre? Some are used to describe cultural roots, but a lot of them we as markers for classifying styles of art and lituture.

## The problem:
Making and recording music is the most accessible it has ever been; there's definitely way more music than what is owned by major record labels. Is it possible to create a recommendation engine that will give a user good recommendations of less popular artists?

## The application:
Since it is not likely that independent artists will be paid fairly anytime soon, the quickest way to help out music creators would be to help increase their plays and exposure. The purpose of this project is to help music lovers find something genuinely new, and for independent artists to get more plays and exposure with a slightly bigger paycheck.

## The approach
Using a list of genres I scraped from a few blogs, I searched the Spotify API for the top ten artists in each genre, and saved their genre groupings. Using the artist index from bandcamp.com -- a very large community of independent artists and labels -- I used the Spotify API to confirm which ones were on Spotify, and saved their info in an RDS database on AWS for later use.

In [1]:
import pandas as pd
import spotipy
import json
import curation_station as cs
import functions as f
import pickle
from tqdm import tqdm

https://accounts.spotify.com/authorize?client_id=b25032de1b3c444da54990d230ad65c1&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%2F&scope=user-top-read
Paste the above link into your browser, then paste the redirect url here: http://localhost/?code=AQC5gTi7Wi-VQ8XAe4Ma8MdbD49R7n3p3mbXHYZ8WoIdFaAH7Z9uTLEqaIvbirrn4Fz0WU1JkPXMHjvm46JkbCGZZgi7kx3oUc1YsPJ0GRY30UbVohyx3qIezfWWLb37jenQoMzvOwkES1lQ4JVUwiPRN9W-i4p_YIUXrkpMKdJoWu3sY33Fa5OUtN6i_MxJB0OP50urI70


In [2]:
def merge_dfs(dict_of_dfs):
    log = list(dict_of_dfs.keys())
    starter_df = pd.DataFrame(dict_of_dfs[log[0]]).reset_index().rename(columns={'index':'genre'})
    starter_df['genre'] = log[0]
    for i in tqdm(range(1,len(log))):
        df = pd.DataFrame(dict_of_dfs[log[i]]).reset_index().rename(columns={'index':'genre'})
        df['genre'] = log[i]
        starter_df = pd.concat([starter_df,df],sort=True)
    return starter_df

In [3]:
whatsthis = json.load(open('genre_and_feats_oct18.json','r'))
df = f.merge_dfs(whatsthis)

 65%|██████▍   | 1136/1751 [01:15<01:01,  9.98it/s]

AttributeError: 'NoneType' object has no attribute 'keys'

 65%|██████▍   | 1136/1751 [01:30<01:01,  9.98it/s]

# A whole lotta functions to help sort through the mess

In [3]:
###### cleaning the genre list made from the keys of no/gb genre dict #####
def clean_list(lst):
    clean = []
    for genre in lst:
        genre = genre.lower()
        if " / " in genre:
            clean.append(genre.split(" / ")[1])
            clean.append(genre.split(" / ")[0])
        else:
            clean.append(genre)
    return clean

####### function to flatten out lists of lists (i made a lot of those and working against time) ######
### will fix this during refactoring phase ####

def flatten_lists(list_of_lists):
    return [x for y in list_of_lists for x in y]

''' function that takes in a list of artist album_ids
    and returns audio features of their top tracks '''

def get_top_tracks(artist_id):
    return [x['id'] for x in f.sp.artist_top_tracks(artist_id)['tracks']]

### function takes in a list of genres and returns the top ten artists and their top ten songs from spotify
def spotify_genre_dict(genres):
    case_list = []
    dictionary = {}
    for genre in tqdm(genres):
        open('genre_dicts2.pickle','ab')
        artist_results = f.sp.search(q=f'genre:{genre}',type='artist')['artists']['items']
        artists = [(x['name'],x['id']) for x in artist_results]
        related_genres = [x['genres'] for x in artist_results]
        top_trax = [get_top_tracks(id) for artist,id in artists]
        dictionary = {'genre': genre,
                      'related_genres': related_genres,
                      'artists': [x[0] for x in artists],
                      'artist_ids': [x[1] for x in artists],
                      'top_trax': top_trax
                      }
        case_list.append(dictionary)
        f.refresh_token()
    pickle.dump(case_list,open('genre_dicts2.pickle','wb'))
    return

''' function takes in a list of artists and returns their spotify info in a dictionary '''
def get_artist_info(loa):
    case_list = []
    bc_dict = {}
    for i in tqdm(range(len(loa))):
        open('new_bc_dict.pickle','a')
        try:
            r = f.find_artist(loa[i][0])['artists']['items'][0]
            bc_dict = {'followers': r['followers']['total'],
                       'genres': r['genres'],
                       'id': r['id'],
                       'artist_name': r['name'],
                       'popularity': r['popularity']}
            case_list.append(bc_dict)
            pickle.dump(case_list,open('new_bc_dict.pickle','wb'))
        except:
            print('no longer on spotify')
    return

#### function takes in the list of tuples and creates a new column 'genre' which is = [0] and the data comes from [1] #####
def merge_dfs(list_of_df):
    list_of_genres = [x[0] for x in list_of_df]
    starter_df = list_of_df[0][1].reset_index().rename(columns={'index':'genre'})
    starter_df['genre'] = list_of_genres[0]
    for i in tqdm(range(1,len(list_of_df))):
        df = list_of_df[i][1].reset_index().rename(columns={'index':'genre'})
        df['genre'] = list_of_df[i][0]
        starter_df = pd.concat([starter_df,df])
    return starter_df

def find_genres(artist_list):
    genres = []
    for artist in tqdm(artist_list):
        try:
            r = f.find_artist(artist)
            if len(r['artists']['items']) > 1:
                for results in r['artists']['items']:
                    if len(results['genres']) < 1:
                        continue
                    else:
                        genres.append(results['genres'])
        except:
            print('something dumb happened')
            continue
    return genres

In [4]:
######## load pickles and json
### list of dictionaries -- 'related_genres' key needs to be condensed to a list
kagg_genres = pickle.load(open('genre_dicts2.pickle','rb')) 

### list of tuples (i think) for genres and the audio features -- don't need it now
feets_n_genres = pickle.load(open('features_eda.pickle','rb'))

### dictionaries of bc artists ['genres'] key needs to be put in list (with the condition there is more than 1 genre)
temp_bc_dicts = pickle.load(open('new_bc_dict.pickle','rb'))

### genre dictionary -- probably not up to data -- ignore for now
g_dict = json.load(open('test_genre_dict.json','r'))

## blog data
nd = json.load(open('/home/xristsos/Documents/nodata/bigNoOct7.json','r'))
gb = json.load(open('/home/xristsos/Documents/nodata/glory_oct7.json','r'))
data = nd + gb

In [5]:
yee = pickle.load(open('/media/xristos/USB STICK/new_bc_confirmed.pickle','rb'))
more_genres = [x['genres'] for x in yee if len(x['genres']) > 1]

In [7]:
kagg_genres[0].keys()

dict_keys(['genre', 'related_genres', 'artists', 'artist_ids', 'top_trax'])

In [24]:
ggg = [x['related_genres'] for x in kagg_genres if len(x['related_genres']) > 1]

In [ ]:
# audio_df = merge_dfs(feets_n_genres)
# audio_df.drop(columns='analysis_url',inplace=True)
# audio_df

In [10]:
## remove the duplicates
data = cs.remove_duplicates(data)
g = cs.genre_dict_builder(data)
g = list(g.keys())
ndgb_genres = cs.get_genres(data)
for i in range(len(ndgb_genres)):
    ndgb_genres[i] = clean_list(ndgb_genres[i])
    
## clean the list of genres so there are no special characters
g = clean_list(g)

100%|██████████| 44043/44043 [00:00<00:00, 293857.55it/s]


In [42]:
g

['techno',
 'deep house',
 'electronic',
 'ep',
 'post punk',
 'acid',
 'album',
 'industrial',
 'krautrock',
 'new wave',
 'no wave',
 'breaks',
 'electro',
 'idm',
 'rap',
 'hip hop',
 'experimental',
 'downtempo',
 'noise',
 'ambient',
 'shoegaze',
 'alternative',
 'bass',
 'drums',
 'breakbeat',
 'jazz',
 'field recording',
 'glitch',
 'house',
 'disco',
 'grime',
 'folk',
 'ethereal',
 'piano',
 'psychedelic',
 'avantgarde',
 'free improvisation',
 'modern classical',
 'dub',
 'drone',
 'mixtape',
 'ebm',
 'world',
 'country',
 'dark ambient',
 'leftfield',
 'minimal',
 'rock',
 'tribal',
 'indie',
 'soul',
 'funk',
 'uk garage',
 'synth wave',
 'punk',
 'reggae',
 'jungle',
 'spoken word',
 'musique concrète',
 'uncategorized',
 'vaporwave',
 'dub techno',
 'darkwave',
 'abstract',
 'anthology',
 'various artists',
 'footwork',
 'drum n bass',
 'compilation',
 'club',
 'instrumental',
 'new age',
 'remixes',
 'synth-pop',
 'single',
 'dubstep',
 'nu disco',
 'space',
 'dance hall

In [ ]:
#### load the dictionaries from spotify ####
# for x in temp_bc_dicts:
#     del x['artists']
#     del x['artist_ids']
# genre_lists = [x['related_genres'] for x in genre_dicts]
# genre_lists = flatten_lists(genre_lists)

In [ ]:
#### setting up features and genres dataframe
for item in feets_n_genres:
    item[1].drop(columns=['track_href','id','type','uri'],inplace=True)
idk = {}
for item in feets_n_genres:
    idk[item[0]] = item[1]

In [ ]:
# new_data = pd.DataFrame.from_csv('/home/xristsos/Documents/nodata/songDb.tsv', sep='\t', encoding = "ISO-8859-1")
# kaggle_artist_list = list(new_data.index)

In [ ]:
# new = find_genres(kaggle_artist_list[:2500])

In [43]:
genre_lists = []
genre_lists = more_genres + g + ggg

In [39]:
### creating a genre dictionary that represents how often a genre appears with another one
g_dict = {}
for genre in tqdm(more_genres):
    for i in range(len(genre)):
        if genre[i-1] not in g_dict:
            g_dict[genre[i-1]] = {}
        if genre[i] not in g_dict[genre[i-1]]:
            g_dict[genre[i-1]][genre[i]] = 0
        g_dict[genre[i-1]][genre[i]] += 1

100%|██████████| 273/273 [00:00<00:00, 61383.35it/s]


In [ ]:
##### dataframed it ####
g_df = pd.DataFrame(g_dict)
g_df.fillna(value=0,inplace=True)

In [28]:


def clean_df(df):
    garbage = ['ep','album','single','compilation','various artists','remixes','promo']
    for col in df.columns:
        if col in garbage:
            df.drop(columns=col,inplace=True)
    for ind in df.index:
        if ind in garbage:
            df.drop(index=ind, inplace=True)

In [29]:
clean_df(g_df)
g_dict = g_df.to_dict()

In [30]:
g_df = pd.DataFrame(g_dict)

In [6]:
def single_genre_feats(genre):
    audio_feats = []
    trax = []
    a_artists = []
    for i in tqdm(range(1000)):
        r = f.sp.search(q=f'genre:{genre}',type='artist',limit=50,offset=i)['artists']['items']
        artists = [(x['name'],x['id']) for x in r]
        top_trax = f.flatten_lists([f.get_top_tracks(id) for artist,id in artists])
        f.refresh_token()
        trax.append(top_trax)
        a_artists.append(artists)
    pickle.dump(trax,open('oct22electronic-search.pickle','wb'))
    for i in tqdm(range(len(trax)/25)):
        audio_feats.append(f.sp.audio_features(trax[:24]))
        f.refresh_token()
    return audio_feats, a_artists

In [20]:
f.refresh_token()

In [21]:
big_ass_list_of_features = []
for i in tqdm(range(0,162700,50)):
    r = f.sp.audio_features(getitdone[i:i+49])
    big_ass_list_of_features.append(r)


 10%|▉         | 314/3254 [00:43<07:32,  6.49it/s]


 19%|█▉        | 618/3254 [01:27<05:45,  7.64it/s]


 28%|██▊       | 923/3254 [02:10<04:55,  7.89it/s]


 38%|███▊      | 1221/3254 [02:54<05:07,  6.62it/s]


 47%|████▋     | 1518/3254 [03:40<04:38,  6.23it/s]


...

 65%|██████▍   | 2110/3254 [05:08<06:51,  2.78it/s]


 74%|███████▍  | 2406/3254 [05:51<02:04,  6.81it/s]


 83%|████████▎ | 2702/3254 [06:36<01:31,  6.03it/s]


 92%|█████████▏| 2998/3254 [07:20<00:35,  7.13it/s]


100%|██████████| 3254/3254 [07:58<00:00,  6.13it/s]

In [23]:
balof = f.flatten_lists(big_ass_list_of_features)

In [24]:
pickle.dump(balof,open('FINAL LIST OF FEATURES FOR ELECTRONIC MUSIC.pickle','wb'))

In [13]:
pickle.dump(getitdone,open('162kUnqieElectronicTrackID.pickle','wb'))

In [17]:
162700/50

3254.0

In [12]:
l2 = []
print(len(getitdone))
for x in tqdm(getitdone):
    if x in l2:
        getitdone.remove(x)
    else:
        l2.append(x)
print(len(getitdone))


  0%|          | 206/318212 [00:00<02:35, 2041.07it/s]

318212



  4%|▍         | 12647/318212 [00:29<13:20, 381.50it/s]


  7%|▋         | 22386/318212 [00:59<16:12, 304.13it/s]


 10%|▉         | 30575/318212 [01:28<18:12, 263.24it/s]


...

 49%|████▊     | 154797/318212 [16:20<25:41, 106.02it/s]


 50%|████▉     | 157954/318212 [16:50<27:23, 97.49it/s]


 51%|█████     | 161096/318212 [17:21<25:28, 102.76it/s]


 51%|█████     | 162703/318212 [17:36<16:50, 153.95it/s]

162703


In [51]:
test_list = []
counter = 0
for x in tqdm(range(0,487900,50)):
    r = f.sp.tracks(getitdone[x:x+49])
    test_list.append([x['name'] for x in r['tracks']])
    counter += 1
    if counter == 100:
        f.refresh_token()
        counter = 0
    else:
        continue


  3%|▎         | 303/9758 [01:06<47:40,  3.31it/s]


  6%|▌         | 605/9758 [02:13<21:09,  7.21it/s]


  9%|▉         | 907/9758 [03:01<23:20,  6.32it/s]


 12%|█▏        | 1205/9758 [03:47<21:23,  6.66it/s]


 15%|█▌        | 1501/9758 [04:34<28:22,  4.85it/s]


...

retrying ...2secs



 94%|█████████▍| 9213/9758 [27:15<02:14,  4.05it/s]

retrying ...2secs



 97%|█████████▋| 9509/9758 [28:21<00:42,  5.85it/s]


100%|██████████| 9758/9758 [29:17<00:00,  5.73it/s]

In [9]:
len(getitdone)

487908

In [7]:
feets,artists = single_genre_feats('electronic')

100%|██████████| 1000/1000 [2:17:22<00:00,  7.79s/it] 


TypeError: 'float' object cannot be interpreted as an integer

In [6]:
getitdone = pickle.load(open('oct22electronic-search.pickle','rb'))

In [7]:
getitdone = [item for sublist in getitdone for item in sublist]

In [46]:
first1000 = getitdone[:1000]
len(getitdone)

487908

In [47]:
487900/50

9758.0

In [40]:
f.sp.tracks(first1000[:5])['tracks']

[{'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/540vIaP2JwjQb9dm3aArA4'},
     'href': 'https://api.spotify.com/v1/artists/540vIaP2JwjQb9dm3aArA4',
     'id': '540vIaP2JwjQb9dm3aArA4',
     'name': 'DJ Snake',
     'type': 'artist',
     'uri': 'spotify:artist:540vIaP2JwjQb9dm3aArA4'}],
   'available_markets': ['AD',
    'AE',
    'AR',
    'AT',
    'AU',
    'BE',
    'BG',
    'BH',
    'BO',
    'BR',
    'CA',
    'CH',
    'CL',
    'CO',
    'CR',
    'CY',
    'CZ',
    'DE',
    'DK',
    'DO',
    'DZ',
    'EC',
    'EE',
    'EG',
    'ES',
    'FI',
    'FR',
    'GB',
    'GR',
    'GT',
    'HK',
    'HN',
    'HU',
    'ID',
    'IE',
    'IL',
    'IN',
    'IS',
    'IT',
    'JO',
    'KW',
    'LB',
    'LI',
    'LT',
    'LU',
    'LV',
    'MA',
    'MC',
    'MT',
    'MX',
    'MY',
    'NI',
    'NL',
    'NO',
    'NZ',
    'OM',
    'PA',
    'PE',
    'PH',
    'PL',
    'PS',
    'PT',
    'P

In [50]:
f.refresh_token()

In [52]:
ggg = f.flatten_lists(test_list)

478142

In [60]:
len(ggg)

154466

In [61]:
478142-154466

323676

In [2]:
ggg = pickle.load(open('sorting through duplicates needs work in the notebook.pickl','rb'))

In [3]:
l1 = []
print(len(ggg))
for x in tqdm(ggg):
    if x not in l1:
        l1.append(x)
    else:
        ggg.remove(x)
print(len(l1))
print(len(ggg))

  0%|          | 243/154466 [00:00<01:03, 2413.34it/s]

154466


 53%|█████▎    | 81619/154466 [03:49<03:25, 354.93it/s]

8772
81619


In [59]:
pickle.dump(ggg,open('sorting through duplicates needs work in the notebook.pickl','wb'))

'Loco Contigo (feat. J. Balvin & Tyga)'

## Sources:
the world’s biggest four music labels responsible for 87% of content available on Spotify.
-https://www.businessofapps.com/data/spotify-statistics/

75% of music from streaming https://www.theverge.com/2018/9/20/17883584/streaming-record-sales-music-industry-revenue